In [1]:
import yaml
import spotipy
import urllib as url
import random
import pickle
from spotipy.oauth2 import SpotifyOAuth

In [2]:
creds = yaml.load(open('/Users/daverappeport/.spotify/creds.yml','r'), Loader=yaml.FullLoader)
redirect_url="http://localhost:9001/callback"
scope = ["user-library-read", "playlist-modify-private","playlist-modify-public"]

In [3]:
auth_manager=SpotifyOAuth(client_id=creds.get('client_id'), client_secret=creds.get('client_secret'), redirect_uri=redirect_url, scope=scope)
sp=spotipy.Spotify(auth_manager=auth_manager)

In [4]:
results = sp.current_user_saved_tracks()

In [5]:
results.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [6]:
results.get('items')[0].get('track').keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [7]:
user=sp.current_user()

In [8]:
for plist in sp.user_playlists(user=user['id']).get('items'):
    print(plist.get('name'))

My Peloton Music by RaveDapeport
Golden Hour
Rapp Haus
Ghost Town
Asian Dinner
Vibes
Cruise
The Long and Winding Road
good shit
Workout
Starred
Gimme Some More
Liked from Radio
GDTG


In [9]:
running_seeds_url="https://open.spotify.com/playlist/5NrLk0qUyICYEG8cZWK7SL"
res_playlist_id=url.parse.urlparse(running_seeds_url).path.split('/')[2]

In [10]:
running_recs_url="https://open.spotify.com/playlist/3Elnf5eDmqXA9dC52Dr9UJ"
rec_playlist_id=url.parse.urlparse(running_recs_url).path.split('/')[2]

In [11]:
seed_tracks = sp.playlist_items(playlist_id=res_playlist_id)

In [12]:
seed_track_ids = [track.get('track').get('id') for track in seed_tracks.get('items')]

In [13]:
seed_track_ids

['6rx7O4uhp7mPfNvq9FCkos',
 '7MkZtFx58ZUBF4RqwOBQwk',
 '6Nn74LbI9LzxgHci3rDQ6w',
 '5v4vZzXgcvSzFk9Ftlc1OP',
 '3WqifovLWUQICYOAad0aNP']

In [14]:
res = sp.recommendations(seed_tracks=seed_track_ids)
len(res.get('tracks'))

20

In [15]:
rec_track_ids = [track.get('id') for track in res.get('tracks')]

In [16]:
for track in res.get('tracks'):
    print(track.get('name'), ',', track.get('artists')[0].get('name'))

I Got You , Disciples
Way It Is , Mr. Belt & Wezol
On My Mind , Circles Around The Sun
Tongue N' Groove , Steve Kimock Band
So Flute , St Germain
Crush , Campsite Dream
Lift You Up , Sean Koch
Warm , SG Lewis
Helplessly Hoping - 2005 Remaster , Crosby, Stills & Nash
Acrobatic , SIDEPIECE
Ritual Union , Little Dragon
Just A Feeling , Phantoms
Alive , Zeds Dead
Need You - Radio Edit , Sons Of Maria
Casey Jones - 2013 Remaster , Grateful Dead
Stay For It , RL Grime
Let Go , RAC
Awaken , Big Wild
Fifteen - Oxford Remix , Goldroom
Never Lost , Amtrac


In [17]:
sp.playlist_replace_items(playlist_id=rec_playlist_id, items=rec_track_ids)

{'snapshot_id': 'NSw4Yzg3YzU5ODY1Mjk5NGEzYWQ5ZDc2NDAyZjg2YzEwMzFjNTUxYjk5'}

### if you need to clear the playlist
sp.playlist_remove_all_occurrences_of_items(
    playlist_id=rec_playlist_id, 
    items=[track.get('track').get('id') for track in sp.playlist_items(playlist_id=rec_playlist_id).get('items')]
)

In [18]:
# audio features
sp.audio_features(tracks=rec_track_ids)[:3]

[{'danceability': 0.711,
  'energy': 0.876,
  'key': 5,
  'loudness': -4.144,
  'mode': 1,
  'speechiness': 0.0551,
  'acousticness': 0.145,
  'instrumentalness': 0,
  'liveness': 0.116,
  'valence': 0.706,
  'tempo': 122.969,
  'type': 'audio_features',
  'id': '5DF2KFDVeC7wF5CXipl9WN',
  'uri': 'spotify:track:5DF2KFDVeC7wF5CXipl9WN',
  'track_href': 'https://api.spotify.com/v1/tracks/5DF2KFDVeC7wF5CXipl9WN',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5DF2KFDVeC7wF5CXipl9WN',
  'duration_ms': 210732,
  'time_signature': 4},
 {'danceability': 0.868,
  'energy': 0.69,
  'key': 3,
  'loudness': -5.606,
  'mode': 0,
  'speechiness': 0.0856,
  'acousticness': 0.0227,
  'instrumentalness': 0.0526,
  'liveness': 0.059,
  'valence': 0.457,
  'tempo': 123.997,
  'type': 'audio_features',
  'id': '2dwnD6tMYqp8zQmfg1XvNQ',
  'uri': 'spotify:track:2dwnD6tMYqp8zQmfg1XvNQ',
  'track_href': 'https://api.spotify.com/v1/tracks/2dwnD6tMYqp8zQmfg1XvNQ',
  'analysis_url': 'https://api.s

In [19]:
#### let's get my 200 most recent saved tracks
saved_tracks = []

for n in range(0,6):
    tracks = sp.current_user_saved_tracks(limit=50, offset=n)
    saved_tracks = saved_tracks + tracks.get('items')

print(len(saved_tracks))

300


In [20]:
??sp.current_user_saved_tracks

Signature: sp.current_user_saved_tracks(limit=20, offset=0, market=None)
Source:   
    def current_user_saved_tracks(self, limit=20, offset=0, market=None):
        """ Gets a list of the tracks saved in the current authorized user's
            "Your Music" library

            Parameters:
                - limit - the number of tracks to return
                - offset - the index of the first track to return
                - market - an ISO 3166-1 alpha-2 country code

        """
        return self._get("me/tracks", limit=limit, offset=offset, market=market)
File:      ~/venv/dlrappeport/lib/python3.8/site-packages/spotipy/client.py
Type:      method


In [21]:
# res.get('items')[0].get('track')
new_music = []

for item in saved_tracks:
    item = item.get('track')
    # print(item.get('name') + ';' + item.get('artists')[0].get('name'))
    
    seed_id = item.get('id')
    recs = sp.recommendations(seed_tracks=[seed_id])
    
    for track in recs.get('tracks'):
        new_music.append(track)

In [22]:
# and now we have 4k new tracks
print(len(new_music))

new_music[0]

6000


{'album': {'album_type': 'SINGLE',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/23BtKJexFLn0Ri6s7BEC4d'},
    'href': 'https://api.spotify.com/v1/artists/23BtKJexFLn0Ri6s7BEC4d',
    'id': '23BtKJexFLn0Ri6s7BEC4d',
    'name': 'Vijay & Sofia Zlatko',
    'type': 'artist',
    'uri': 'spotify:artist:23BtKJexFLn0Ri6s7BEC4d'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BZ',
   'CA',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
   'ID',
   'IE',
   'IL',

In [23]:
new_music_dict = {}

for m in new_music:
    new_music_dict[m.get('id')] = m
    
# get audio features
for n in range(0, int(len(new_music)/50)):
    features = sp.audio_features(tracks=[t.get('id') for t in new_music[n*50:(n+1)*50]])
    
    for f in features:
        new_music_dict[f.get('id')]['audio_features'] = f

In [24]:
k = random.choice(list(new_music_dict.keys()))
new_music_dict[k]

{'album': {'album_type': 'SINGLE',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2vf4pRsEY6LpL5tKmqWb64'},
    'href': 'https://api.spotify.com/v1/artists/2vf4pRsEY6LpL5tKmqWb64',
    'id': '2vf4pRsEY6LpL5tKmqWb64',
    'name': 'Elderbrook',
    'type': 'artist',
    'uri': 'spotify:artist:2vf4pRsEY6LpL5tKmqWb64'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
   'ID',


In [25]:
# save spotify data to pickle file    
with open('new_music_dict.pickle', 'wb') as handle:
    pickle.dump(new_music_dict, handle)

In [26]:
total_ms = 0
for k in new_music_dict.keys():
    total_ms+=new_music_dict[k]['audio_features']['duration_ms']

total_hours = total_ms/1e3/60/60
print(f'We have {total_hours} of music in our data')

We have 286.10810472222227 of music in our data


In [27]:
len(new_music_dict.keys())

4219